In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [ ]:
import numba
from numba import jit

In [ ]:
Lx, Ly = 1.0, 1.0
nx, ny = 41, 41

x = numpy.linspace(0.0, Lx, num=nx)
y = numpy.linspace(0.0, Ly, num=ny)

dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

Re = 10

dt = 1e-4

nt = 9000

In [ ]:
u0 = numpy.zeros((ny, nx))
u0[-1, :] = 1
v0 = numpy.zeros((ny, nx))
p0 = numpy.zeros((ny, nx))

In [ ]:
@jit(nopython=True)
def fractional_step(u0, v0, p0, dt, dx, Re, nt, maxiter=20000, rtol=1e-6):
    u = u0.copy()
    v = v0.copy()
    u_star = u.copy()
    v_star = v.copy()
    b = u0.copy()
    p = p0.copy()
    for n in range(nt):
        u_star[1:-1, 1:-1] = ( u[1:-1, 1:-1] + 
                               dt * (- u[1:-1, 1:-1] * (u[1:-1, 2:] - u[1:-1, :-2]) / (2*dx)
                                     - v[1:-1, 1:-1] * (u[2:, 1:-1] - u[:-2, 1:-1]) / (2*dx)
                                     + (1/Re) * ( (u[1:-1, :-2] - 2 * u[1:-1, 1:-1] + u[1:-1, 2:]) / (dx**2) +
                                                  (u[:-2, 1:-1] - 2 * u[1:-1, 1:-1] + u[2:, 1:-1]) / (dx**2) )) )
        v_star[1:-1, 1:-1] = ( v[1:-1, 1:-1] + 
                               dt * (- u[1:-1, 1:-1] * (v[1:-1, 2:] - v[1:-1, :-2]) / (2*dx)
                                     - v[1:-1, 1:-1] * (v[2:, 1:-1] - v[:-2, 1:-1]) / (2*dx)
                                     + (1/Re) * ( (v[1:-1, :-2] - 2 * v[1:-1, 1:-1] + v[1:-1, 2:]) / dx**2 +
                                                  (v[:-2, 1:-1] - 2 * v[1:-1, 1:-1] + v[2:, 1:-1]) / (dx**2) )) )
        
                
        b[1:-1, 1:-1] = 1/dt * ( (u_star[1:-1, 2:] - u_star[1:-1, :-2]) / (2*dx) +
                                 (v_star[2:, 1:-1] - v_star[:-2, 1:-1]) / (2*dx) )

        
        r = numpy.zeros_like(p)
        Ad = numpy.zeros_like(p)
        conv = []
        diff = rtol + 1
        ite = 0
        r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
        d = r.copy()
        while diff > rtol and ite < maxiter:
            pk = p.copy()
            rk = r.copy()
            Ad[1:-1, 1:-1] = A(d)
            alpha = numpy.sum(r * r) / numpy.sum(d * Ad)
            p = pk + alpha * d
            r = rk - alpha * Ad
            beta = numpy.sum(r * r) / numpy.sum(rk * rk)
            d = r + beta * d
            diff = l2_norm(p, pk)
            conv.append(diff)
            ite += 1


#         u[1:-1, 1:-1] = u_star[1:-1, 1:-1] - (p[1:-1, 2:] - p[1:-1, :-2]) / (2*dx)
#         v[1:-1, 1:-1] = v_star[1:-1, 1:-1] - (p[2:, 1:-1] - p[:-2, 1:-1]) / (2*dx)
        
#         u[0, :] = 0
#         v[0, :] = 0
#         #p[0, 2:] = p[0, :-2]
#         p[0, :] = 0
#         u[:, 0] = 0
#         v[:, 0] = 0
#         #p[2:, 0] = p[:-2, 0]
#         p[:, 0] = 0
#         u[:, -1] = 0
#         v[:, -1] = 0
#         #p[2:, -1] = p[:-2, -1]
#         p[:, -1] = 0
#         u[-1, :] = 1
#         v[-1, :] = 0
#         p[-1, :] = 0
    
    
    return u, v, p

In [ ]:
u, v, p = fractional_step(u0, v0, p0, dt, dx, Re, nt, maxiter=20000, rtol=1e-6)

In [ ]:
p